# CIFAR-10

La première étape intermédiaire de notre projet est d'utiliser les algorithmes ci-dessous sur le célébre dataset CIFAR-10.

Les algorithmes à étudier :

**Modèles précédents**
- Modèle Linéaire
- Perceptron Multicouches

**Nouveaux modèles**
- ConvNet(s)
- ResNets / HighwayNets 
- RNN(s)

Pour chacun des algorithmes cités, il faut :
1. L'influence de tous les hyperparamètres des modèles
    - Structure
    - Fonctions d'activations
    - etc.
2. Les paramètres des algorithmes d'apprentissages
    - Learning Rate
    - Momentum
    - etc.
    
----

### Méthodologie
1. Créer un modèle classique.
2. Entraîner le modèle pendant 500 epochs.
3. Examiner sa courbe TensorBoard et son accuracy.
4. Augmenter un des hyperparamètre.
5. Réduire ce même hyperparamètre.
6. Dire l'influence de cette hyperparamètre sur la courbe et sur le modèle de manière générale.
7. Recommencer à partir de l'étape 3 pour tout les hyperparamètres possibles.

#### Hyperparamètres
- Batch size
- Learning rate
- Momentum
- Structure
- Fonction d'activation
- Initialization de kernel
- Regularizers (Dropout, L1 Norm, L2 Norm)

#### Qui s'occupe de quoi ?
- Perceptron (Mamadian)
- MLP (Réda M.)
- ConvNet (Reda B.)

In [21]:
import os
import numpy as np
from numpy.random import seed
import tensorflow as tf
from tensorflow.keras.layers import Flatten, Dense, Conv2D, BatchNormalization, Input, Average, MaxPool2D
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.metrics import categorical_accuracy
from tensorflow.keras.optimizers import SGD,Adam,RMSprop
from tensorflow.keras.activations import relu, softmax, tanh, sigmoid
from tensorflow.keras.initializers import he_normal, glorot_uniform
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.models import Model, load_model , Sequential
from tensorflow.keras.regularizers import l2, l1, l1_l2
from tensorflow.random import set_seed

In [2]:
print("Version de TensorFlow :", tf.__version__)
print("Nom du GPU :", tf.test.gpu_device_name())

tf.keras.backend.clear_session()
tf.config.optimizer.set_jit(False)

Version de TensorFlow : 2.7.0
Nom du GPU : 


## Importation du dataset

In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data() 

In [4]:
NUM_CLASSES = 10
IMG_SIZE = x_train[0].shape

In [5]:
x_train.shape

(50000, 32, 32, 3)

In [6]:
x_train = x_train.astype('float32') / 256
x_test = x_test.astype('float32') / 256

y_train = to_categorical(y_train, num_classes=NUM_CLASSES)
y_test = to_categorical(y_test, num_classes=NUM_CLASSES)

In [7]:
LOG_DIR = os.path.join("logs") 
MODEL_DIR = os.path.join("models") 

## Fixer les seeds

In [8]:
set_seed(42) # TensorFlow
seed(42) # NumPy

# ConvNet

## Learning rate et Momentum 
### Hyperparamètre : 
#### NUM_LAYERS = [5,10,15]
#### OPTIMIZER = SGD
#### BATCH_SIZE = 512
#### EPOCHS = 200 
#### learning_rates = [0.5, 0.1, 0.05,0 .01, 0.005, 0.001]
#### momentums = [0.25, 0.5, 0.9]

In [9]:
NUM_LAYERS = list(range(3, 6))
OPTIMIZER = SGD
BATCH_SIZE = 512
EPOCHS = 100
SHUFFLE = True

In [10]:
learning_rates = [0.5, 0.1, 0.05,0.01, 0.005, 0.001]
momentums = [0.25, 0.5, 0.9]


def ConvNet(num_layer, filters_by_layers):
    input_layer = Input(shape=(32, 32, 3))
    hidden_layers = input_layer
    
    for n in range(num_layer):
        hidden_layers = Conv2D(filters_by_layers[n], (3, 3), padding="same", activation=tanh)(hidden_layers)
        hidden_layers = MaxPool2D()(hidden_layers)
    
    hidden_layers = Flatten()(hidden_layers)
    output_layer = Dense(NUM_CLASSES, activation=softmax)(hidden_layers)
    return Model(input_layer, output_layer)


for lr in learning_rates:
    for mom in momentums:
        num_layer = int(np.random.choice(NUM_LAYERS, 1))
        filters_by_layers = [32*x for x in range(1, num_layer+1)]

        convnet = ConvNet(num_layer, filters_by_layers)
        convnet.compile(loss=categorical_crossentropy,
                    optimizer=SGD(lr, momentum=mom),
                    metrics=categorical_accuracy)
        CONVNET_LOG = os.path.join(LOG_DIR, "convnet",
                        f"convnet_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_SGD_lr_{lr}_mom_{mom}_layers_{'_'.join(str(e) for e in filters_by_layers)}_af_tanh")
        CONVNET_MODEL = os.path.join(MODEL_DIR, "convnet",
                        f"convnet_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_SGD_lr_{lr}_mom_{mom}_layers_{'_'.join(str(e) for e in filters_by_layers)}_af_tanh.keras")
        convnet.fit(x_train,
                    y_train,
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    validation_data=(x_test, y_test),
                    shuffle=SHUFFLE,
                    callbacks=[tf.keras.callbacks.TensorBoard(CONVNET_LOG, histogram_freq=1)] 
                   ) 
        convnet.save(CONVNET_MODEL)
                       

Epoch 1/100
98/98 [==============================] - 64s 650ms/step - loss: 27.4190 - categorical_accuracy: 0.1034 - val_loss: 47.0059 - val_categorical_accuracy: 0.1000
Epoch 2/100
98/98 [==============================] - 64s 658ms/step - loss: 41.6136 - categorical_accuracy: 0.1016 - val_loss: 36.8040 - val_categorical_accuracy: 0.1000
Epoch 3/100
98/98 [==============================] - 64s 657ms/step - loss: 39.1387 - categorical_accuracy: 0.1015 - val_loss: 42.8314 - val_categorical_accuracy: 0.1000
Epoch 4/100
98/98 [==============================] - 64s 658ms/step - loss: 42.8203 - categorical_accuracy: 0.1006 - val_loss: 45.7808 - val_categorical_accuracy: 0.1000
Epoch 5/100
98/98 [==============================] - 65s 660ms/step - loss: 41.6884 - categorical_accuracy: 0.0990 - val_loss: 38.4759 - val_categorical_accuracy: 0.1000
Epoch 6/100
98/98 [==============================] - 66s 669ms/step - loss: 39.2255 - categorical_accuracy: 0.1010 - val_loss: 38.5461 - val_categoric

Epoch 49/100
98/98 [==============================] - 65s 661ms/step - loss: 40.2993 - categorical_accuracy: 0.1013 - val_loss: 36.7473 - val_categorical_accuracy: 0.1000
Epoch 50/100
98/98 [==============================] - 65s 661ms/step - loss: 39.0294 - categorical_accuracy: 0.1023 - val_loss: 35.6384 - val_categorical_accuracy: 0.1000
Epoch 51/100
98/98 [==============================] - 65s 663ms/step - loss: 41.5821 - categorical_accuracy: 0.0996 - val_loss: 36.5046 - val_categorical_accuracy: 0.1000
Epoch 52/100
98/98 [==============================] - 65s 664ms/step - loss: 40.0535 - categorical_accuracy: 0.0981 - val_loss: 23.3421 - val_categorical_accuracy: 0.1000
Epoch 53/100
98/98 [==============================] - 67s 685ms/step - loss: 38.6146 - categorical_accuracy: 0.0996 - val_loss: 46.9150 - val_categorical_accuracy: 0.1000
Epoch 54/100
98/98 [==============================] - 65s 665ms/step - loss: 40.7456 - categorical_accuracy: 0.0988 - val_loss: 43.1247 - val_cat

Epoch 97/100
98/98 [==============================] - 64s 655ms/step - loss: 40.9864 - categorical_accuracy: 0.1001 - val_loss: 41.0466 - val_categorical_accuracy: 0.1000
Epoch 98/100
98/98 [==============================] - 64s 657ms/step - loss: 41.7002 - categorical_accuracy: 0.0991 - val_loss: 42.2017 - val_categorical_accuracy: 0.1000
Epoch 99/100
98/98 [==============================] - 64s 655ms/step - loss: 39.1299 - categorical_accuracy: 0.0981 - val_loss: 40.6913 - val_categorical_accuracy: 0.1000
Epoch 100/100
98/98 [==============================] - 64s 656ms/step - loss: 41.6182 - categorical_accuracy: 0.1009 - val_loss: 42.7886 - val_categorical_accuracy: 0.1000
Epoch 1/100
98/98 [==============================] - 54s 551ms/step - loss: 488.8546 - categorical_accuracy: 0.1026 - val_loss: 614.1821 - val_categorical_accuracy: 0.1000
Epoch 2/100
98/98 [==============================] - 54s 548ms/step - loss: 539.9284 - categorical_accuracy: 0.0997 - val_loss: 654.7571 - val_

98/98 [==============================] - 54s 548ms/step - loss: 537.2362 - categorical_accuracy: 0.0998 - val_loss: 438.1035 - val_categorical_accuracy: 0.1000
Epoch 45/100
98/98 [==============================] - 53s 542ms/step - loss: 540.6603 - categorical_accuracy: 0.0985 - val_loss: 532.8906 - val_categorical_accuracy: 0.1000
Epoch 46/100
98/98 [==============================] - 53s 541ms/step - loss: 541.7073 - categorical_accuracy: 0.0988 - val_loss: 703.7552 - val_categorical_accuracy: 0.1000
Epoch 47/100
98/98 [==============================] - 53s 545ms/step - loss: 530.8641 - categorical_accuracy: 0.0991 - val_loss: 719.5969 - val_categorical_accuracy: 0.1000
Epoch 48/100
98/98 [==============================] - 54s 547ms/step - loss: 536.4974 - categorical_accuracy: 0.1021 - val_loss: 446.3437 - val_categorical_accuracy: 0.1000
Epoch 49/100
98/98 [==============================] - 53s 540ms/step - loss: 536.4097 - categorical_accuracy: 0.0986 - val_loss: 702.1070 - val_cate

98/98 [==============================] - 53s 542ms/step - loss: 536.7456 - categorical_accuracy: 0.0997 - val_loss: 367.2885 - val_categorical_accuracy: 0.1000
Epoch 92/100
98/98 [==============================] - 53s 539ms/step - loss: 539.9691 - categorical_accuracy: 0.0983 - val_loss: 450.8506 - val_categorical_accuracy: 0.1000
Epoch 93/100
98/98 [==============================] - 53s 541ms/step - loss: 538.9727 - categorical_accuracy: 0.1004 - val_loss: 386.3295 - val_categorical_accuracy: 0.1000
Epoch 94/100
98/98 [==============================] - 53s 541ms/step - loss: 540.4069 - categorical_accuracy: 0.1007 - val_loss: 402.9983 - val_categorical_accuracy: 0.1000
Epoch 95/100
98/98 [==============================] - 53s 542ms/step - loss: 539.5562 - categorical_accuracy: 0.0974 - val_loss: 658.4475 - val_categorical_accuracy: 0.1000
Epoch 96/100
98/98 [==============================] - 53s 540ms/step - loss: 536.9028 - categorical_accuracy: 0.0986 - val_loss: 696.9991 - val_cate

...

98/98 [==============================] - 55s 566ms/step - loss: 1.7173 - categorical_accuracy: 0.4010 - val_loss: 1.7059 - val_categorical_accuracy: 0.4090
Epoch 80/100
98/98 [==============================] - 55s 566ms/step - loss: 1.7137 - categorical_accuracy: 0.4024 - val_loss: 1.7023 - val_categorical_accuracy: 0.4110
Epoch 81/100
98/98 [==============================] - 55s 565ms/step - loss: 1.7101 - categorical_accuracy: 0.4033 - val_loss: 1.6987 - val_categorical_accuracy: 0.4111
Epoch 82/100
98/98 [==============================] - 55s 566ms/step - loss: 1.7066 - categorical_accuracy: 0.4038 - val_loss: 1.6953 - val_categorical_accuracy: 0.4135
Epoch 83/100
98/98 [==============================] - 55s 566ms/step - loss: 1.7031 - categorical_accuracy: 0.4048 - val_loss: 1.6919 - val_categorical_accuracy: 0.4126
Epoch 84/100
98/98 [==============================] - 56s 570ms/step - loss: 1.6996 - categorical_accuracy: 0.4059 - val_loss: 1.6894 - val_categorical_accuracy: 0.4111

98/98 [==============================] - 62s 636ms/step - loss: 1.9467 - categorical_accuracy: 0.3207 - val_loss: 1.9384 - val_categorical_accuracy: 0.3363
Epoch 28/100
98/98 [==============================] - 62s 636ms/step - loss: 1.9377 - categorical_accuracy: 0.3233 - val_loss: 1.9293 - val_categorical_accuracy: 0.3388
Epoch 29/100
98/98 [==============================] - 62s 633ms/step - loss: 1.9289 - categorical_accuracy: 0.3270 - val_loss: 1.9204 - val_categorical_accuracy: 0.3414
Epoch 30/100
98/98 [==============================] - 62s 632ms/step - loss: 1.9199 - categorical_accuracy: 0.3296 - val_loss: 1.9113 - val_categorical_accuracy: 0.3423
Epoch 31/100
98/98 [==============================] - 62s 634ms/step - loss: 1.9111 - categorical_accuracy: 0.3320 - val_loss: 1.9025 - val_categorical_accuracy: 0.3450
Epoch 32/100
98/98 [==============================] - 62s 632ms/step - loss: 1.9023 - categorical_accuracy: 0.3349 - val_loss: 1.8935 - val_categorical_accuracy: 0.3494

98/98 [==============================] - 62s 632ms/step - loss: 1.6353 - categorical_accuracy: 0.4190 - val_loss: 1.6273 - val_categorical_accuracy: 0.4256
Epoch 76/100
98/98 [==============================] - 63s 639ms/step - loss: 1.6314 - categorical_accuracy: 0.4202 - val_loss: 1.6236 - val_categorical_accuracy: 0.4256
Epoch 77/100
98/98 [==============================] - 64s 652ms/step - loss: 1.6276 - categorical_accuracy: 0.4225 - val_loss: 1.6199 - val_categorical_accuracy: 0.4279
Epoch 78/100
98/98 [==============================] - 63s 639ms/step - loss: 1.6240 - categorical_accuracy: 0.4231 - val_loss: 1.6173 - val_categorical_accuracy: 0.4299
Epoch 79/100
98/98 [==============================] - 62s 637ms/step - loss: 1.6201 - categorical_accuracy: 0.4239 - val_loss: 1.6126 - val_categorical_accuracy: 0.4309
Epoch 80/100
98/98 [==============================] - 63s 640ms/step - loss: 1.6165 - categorical_accuracy: 0.4253 - val_loss: 1.6090 - val_categorical_accuracy: 0.4316

98/98 [==============================] - 62s 632ms/step - loss: 1.5292 - categorical_accuracy: 0.4552 - val_loss: 1.5162 - val_categorical_accuracy: 0.4612
Epoch 24/100
98/98 [==============================] - 62s 633ms/step - loss: 1.5142 - categorical_accuracy: 0.4597 - val_loss: 1.5088 - val_categorical_accuracy: 0.4641
Epoch 25/100
98/98 [==============================] - 62s 632ms/step - loss: 1.4992 - categorical_accuracy: 0.4648 - val_loss: 1.4877 - val_categorical_accuracy: 0.4694
Epoch 26/100
98/98 [==============================] - 62s 635ms/step - loss: 1.4860 - categorical_accuracy: 0.4681 - val_loss: 1.4738 - val_categorical_accuracy: 0.4733
Epoch 27/100
98/98 [==============================] - 62s 631ms/step - loss: 1.4708 - categorical_accuracy: 0.4755 - val_loss: 1.4603 - val_categorical_accuracy: 0.4774
Epoch 28/100
98/98 [==============================] - 62s 632ms/step - loss: 1.4573 - categorical_accuracy: 0.4795 - val_loss: 1.4508 - val_categorical_accuracy: 0.4829

98/98 [==============================] - 62s 635ms/step - loss: 1.1313 - categorical_accuracy: 0.6055 - val_loss: 1.1631 - val_categorical_accuracy: 0.5898
Epoch 72/100
98/98 [==============================] - 62s 635ms/step - loss: 1.1258 - categorical_accuracy: 0.6069 - val_loss: 1.1631 - val_categorical_accuracy: 0.5890
Epoch 73/100
98/98 [==============================] - 62s 635ms/step - loss: 1.1205 - categorical_accuracy: 0.6082 - val_loss: 1.1528 - val_categorical_accuracy: 0.5930
Epoch 74/100
98/98 [==============================] - 62s 636ms/step - loss: 1.1156 - categorical_accuracy: 0.6102 - val_loss: 1.1596 - val_categorical_accuracy: 0.5911
Epoch 75/100
98/98 [==============================] - 62s 636ms/step - loss: 1.1089 - categorical_accuracy: 0.6137 - val_loss: 1.1545 - val_categorical_accuracy: 0.5934
Epoch 76/100
98/98 [==============================] - 62s 634ms/step - loss: 1.1038 - categorical_accuracy: 0.6150 - val_loss: 1.1394 - val_categorical_accuracy: 0.5968

In [11]:
from tensorflow.keras.optimizers import SGD,Adam,RMSprop
from tensorflow.keras.activations import relu, softmax, tanh, sigmoid

### Optimizers 
Adam RMSprop SGD

In [13]:
NUM_LAYERS = list(range(3, 6))
OPTIMIZER = [SGD,Adam,RMSprop]
BATCH_SIZE = 512
EPOCHS = 100
SHUFFLE = True
learning_rates = 0.001
momentums = 0.5


def ConvNet(num_layer, filters_by_layers):
    input_layer = Input(shape=(32, 32, 3))
    hidden_layers = input_layer
    
    for n in range(num_layer):
        hidden_layers = Conv2D(filters_by_layers[n], (3, 3), padding="same", activation=tanh)(hidden_layers)
        hidden_layers = MaxPool2D()(hidden_layers)
    
    hidden_layers = Flatten()(hidden_layers)
    output_layer = Dense(NUM_CLASSES, activation=softmax)(hidden_layers)
    return Model(input_layer, output_layer)



for op in OPTIMIZER: 
    if op == SGD:
        num_layer = int(np.random.choice(NUM_LAYERS, 1))
        filters_by_layers = [32*x for x in range(1, num_layer+1)]

        convnet = ConvNet(num_layer, filters_by_layers)
        convnet.compile(loss=categorical_crossentropy,
                    optimizer=SGD(lr, momentum=mom),
                    metrics=categorical_accuracy)
        CONVNET_LOG = os.path.join(LOG_DIR, "convnet",
                        f"convnet_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_SGD_lr_{lr}_mom_{mom}_layers_{'_'.join(str(e) for e in filters_by_layers)}_af_tanh")
        CONVNET_MODEL = os.path.join(MODEL_DIR, "convnet",
                        f"convnet_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_SGD_lr_{lr}_mom_{mom}_layers_{'_'.join(str(e) for e in filters_by_layers)}_af_tanh.keras")
        convnet.fit(x_train,
                    y_train,
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    validation_data=(x_test, y_test),
                    shuffle=SHUFFLE,
                    callbacks=[tf.keras.callbacks.TensorBoard(CONVNET_LOG, histogram_freq=1)] 
                   ) 
        convnet.save(CONVNET_MODEL) 

    elif op == Adam:
        num_layer = int(np.random.choice(NUM_LAYERS, 1))
        filters_by_layers = [32*x for x in range(1, num_layer+1)]

        convnet = ConvNet(num_layer, filters_by_layers)
        convnet.compile(loss=categorical_crossentropy,
                    optimizer=Adam(lr),
                    metrics=categorical_accuracy)
        CONVNET_LOG = os.path.join(LOG_DIR, "convnet",
                        f"convnet_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_Adam_lr_{lr}_mom_{mom}_layers_{'_'.join(str(e) for e in filters_by_layers)}_af_tanh")
        CONVNET_MODEL = os.path.join(MODEL_DIR, "convnet",
                        f"convnet_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_Adam_lr_{lr}_mom_{mom}_layers_{'_'.join(str(e) for e in filters_by_layers)}_af_tanh.keras")
        convnet.fit(x_train,
                    y_train,
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    validation_data=(x_test, y_test),
                    shuffle=SHUFFLE,
                    callbacks=[tf.keras.callbacks.TensorBoard(CONVNET_LOG, histogram_freq=1)] 
                   ) 
        convnet.save(CONVNET_MODEL)
    elif op == RMSprop:
        num_layer = int(np.random.choice(NUM_LAYERS, 1))
        filters_by_layers = [32*x for x in range(1, num_layer+1)]

        convnet = ConvNet(num_layer, filters_by_layers)
        convnet.compile(loss=categorical_crossentropy,
                    optimizer=RMSprop(lr,momentum=mom),
                    metrics=categorical_accuracy)
        CONVNET_LOG = os.path.join(LOG_DIR, "convnet",
                        f"convnet_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_RMSprop_lr_{lr}_mom_{mom}_layers_{'_'.join(str(e) for e in filters_by_layers)}_af_tanh")
        CONVNET_MODEL = os.path.join(MODEL_DIR, "convnet",
                        f"convnet_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_RMSprop_lr_{lr}_mom_{mom}_layers_{'_'.join(str(e) for e in filters_by_layers)}_af_tanh.keras")
        convnet.fit(x_train,
                    y_train,
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    validation_data=(x_test, y_test),
                    shuffle=SHUFFLE,
                    callbacks=[tf.keras.callbacks.TensorBoard(CONVNET_LOG, histogram_freq=1)] 
                   ) 
        convnet.save(CONVNET_MODEL)

Epoch 1/100
98/98 [==============================] - 64s 654ms/step - loss: 2.2875 - categorical_accuracy: 0.1476 - val_loss: 2.2656 - val_categorical_accuracy: 0.1776
Epoch 2/100
98/98 [==============================] - 63s 638ms/step - loss: 2.2435 - categorical_accuracy: 0.2330 - val_loss: 2.2179 - val_categorical_accuracy: 0.2725
Epoch 3/100
98/98 [==============================] - 62s 634ms/step - loss: 2.1847 - categorical_accuracy: 0.2881 - val_loss: 2.1455 - val_categorical_accuracy: 0.3238
Epoch 4/100
98/98 [==============================] - 62s 634ms/step - loss: 2.0971 - categorical_accuracy: 0.3167 - val_loss: 2.0438 - val_categorical_accuracy: 0.3229
Epoch 5/100
98/98 [==============================] - 63s 641ms/step - loss: 2.0011 - categorical_accuracy: 0.3211 - val_loss: 1.9595 - val_categorical_accuracy: 0.3335
Epoch 6/100
98/98 [==============================] - 63s 639ms/step - loss: 1.9363 - categorical_accuracy: 0.3292 - val_loss: 1.9090 - val_categorical_accuracy:

98/98 [==============================] - 62s 632ms/step - loss: 1.2486 - categorical_accuracy: 0.5613 - val_loss: 1.2614 - val_categorical_accuracy: 0.5548
Epoch 50/100
98/98 [==============================] - 63s 646ms/step - loss: 1.2425 - categorical_accuracy: 0.5636 - val_loss: 1.2599 - val_categorical_accuracy: 0.5545
Epoch 51/100
98/98 [==============================] - 63s 642ms/step - loss: 1.2342 - categorical_accuracy: 0.5662 - val_loss: 1.2468 - val_categorical_accuracy: 0.5615
Epoch 52/100
98/98 [==============================] - 62s 632ms/step - loss: 1.2245 - categorical_accuracy: 0.5699 - val_loss: 1.2408 - val_categorical_accuracy: 0.5657
Epoch 53/100
98/98 [==============================] - 62s 633ms/step - loss: 1.2199 - categorical_accuracy: 0.5709 - val_loss: 1.2353 - val_categorical_accuracy: 0.5644
Epoch 54/100
98/98 [==============================] - 62s 632ms/step - loss: 1.2114 - categorical_accuracy: 0.5768 - val_loss: 1.2300 - val_categorical_accuracy: 0.5666

98/98 [==============================] - 62s 633ms/step - loss: 0.9552 - categorical_accuracy: 0.6708 - val_loss: 1.0158 - val_categorical_accuracy: 0.6425
Epoch 98/100
98/98 [==============================] - 62s 634ms/step - loss: 0.9503 - categorical_accuracy: 0.6713 - val_loss: 1.0199 - val_categorical_accuracy: 0.6395
Epoch 99/100
98/98 [==============================] - 62s 634ms/step - loss: 0.9447 - categorical_accuracy: 0.6741 - val_loss: 1.0079 - val_categorical_accuracy: 0.6456
Epoch 100/100
98/98 [==============================] - 62s 636ms/step - loss: 0.9391 - categorical_accuracy: 0.6762 - val_loss: 1.0051 - val_categorical_accuracy: 0.6445
Epoch 1/100
98/98 [==============================] - 64s 649ms/step - loss: 1.7096 - categorical_accuracy: 0.3844 - val_loss: 1.4482 - val_categorical_accuracy: 0.4718
Epoch 2/100
98/98 [==============================] - 62s 636ms/step - loss: 1.3278 - categorical_accuracy: 0.5264 - val_loss: 1.2344 - val_categorical_accuracy: 0.5526


98/98 [==============================] - 62s 638ms/step - loss: 0.0109 - categorical_accuracy: 1.0000 - val_loss: 1.2526 - val_categorical_accuracy: 0.7248
Epoch 46/100
98/98 [==============================] - 62s 635ms/step - loss: 0.0101 - categorical_accuracy: 1.0000 - val_loss: 1.2706 - val_categorical_accuracy: 0.7224
Epoch 47/100
98/98 [==============================] - 62s 635ms/step - loss: 0.0093 - categorical_accuracy: 1.0000 - val_loss: 1.2747 - val_categorical_accuracy: 0.7218
Epoch 48/100
98/98 [==============================] - 62s 633ms/step - loss: 0.0086 - categorical_accuracy: 1.0000 - val_loss: 1.2910 - val_categorical_accuracy: 0.7212
Epoch 49/100
98/98 [==============================] - 62s 635ms/step - loss: 0.0080 - categorical_accuracy: 1.0000 - val_loss: 1.3007 - val_categorical_accuracy: 0.7217
Epoch 50/100
98/98 [==============================] - 62s 635ms/step - loss: 0.0073 - categorical_accuracy: 1.0000 - val_loss: 1.3117 - val_categorical_accuracy: 0.7205

98/98 [==============================] - 62s 634ms/step - loss: 4.5472e-04 - categorical_accuracy: 1.0000 - val_loss: 1.6964 - val_categorical_accuracy: 0.7234
Epoch 94/100
98/98 [==============================] - 63s 641ms/step - loss: 4.2426e-04 - categorical_accuracy: 1.0000 - val_loss: 1.7063 - val_categorical_accuracy: 0.7212
Epoch 95/100
98/98 [==============================] - 62s 636ms/step - loss: 4.0948e-04 - categorical_accuracy: 1.0000 - val_loss: 1.7166 - val_categorical_accuracy: 0.7221
Epoch 96/100
98/98 [==============================] - 62s 637ms/step - loss: 3.8946e-04 - categorical_accuracy: 1.0000 - val_loss: 1.7255 - val_categorical_accuracy: 0.7231
Epoch 97/100
98/98 [==============================] - 62s 635ms/step - loss: 3.6605e-04 - categorical_accuracy: 1.0000 - val_loss: 1.7319 - val_categorical_accuracy: 0.7230
Epoch 98/100
98/98 [==============================] - 62s 634ms/step - loss: 3.4479e-04 - categorical_accuracy: 1.0000 - val_loss: 1.7436 - val_cate

98/98 [==============================] - 56s 572ms/step - loss: 0.5216 - categorical_accuracy: 0.8625 - val_loss: 2.4895 - val_categorical_accuracy: 0.6515
Epoch 42/100
98/98 [==============================] - 57s 579ms/step - loss: 0.5024 - categorical_accuracy: 0.8676 - val_loss: 2.6280 - val_categorical_accuracy: 0.6392
Epoch 43/100
98/98 [==============================] - 56s 573ms/step - loss: 0.5592 - categorical_accuracy: 0.8582 - val_loss: 2.6549 - val_categorical_accuracy: 0.6362
Epoch 44/100
98/98 [==============================] - 56s 574ms/step - loss: 0.5340 - categorical_accuracy: 0.8617 - val_loss: 2.4719 - val_categorical_accuracy: 0.6630
Epoch 45/100
98/98 [==============================] - 56s 573ms/step - loss: 0.5131 - categorical_accuracy: 0.8660 - val_loss: 2.5345 - val_categorical_accuracy: 0.6544
Epoch 46/100
98/98 [==============================] - 56s 574ms/step - loss: 0.5491 - categorical_accuracy: 0.8610 - val_loss: 2.5686 - val_categorical_accuracy: 0.6507

98/98 [==============================] - 57s 578ms/step - loss: 0.6222 - categorical_accuracy: 0.8438 - val_loss: 2.3648 - val_categorical_accuracy: 0.6548
Epoch 90/100
98/98 [==============================] - 57s 582ms/step - loss: 0.6440 - categorical_accuracy: 0.8417 - val_loss: 2.3803 - val_categorical_accuracy: 0.6544
Epoch 91/100
98/98 [==============================] - 56s 575ms/step - loss: 0.6051 - categorical_accuracy: 0.8473 - val_loss: 2.4178 - val_categorical_accuracy: 0.6405
Epoch 92/100
98/98 [==============================] - 56s 571ms/step - loss: 0.6427 - categorical_accuracy: 0.8404 - val_loss: 2.3315 - val_categorical_accuracy: 0.6597
Epoch 93/100
98/98 [==============================] - 56s 572ms/step - loss: 0.6162 - categorical_accuracy: 0.8453 - val_loss: 2.4363 - val_categorical_accuracy: 0.6545
Epoch 94/100
98/98 [==============================] - 56s 570ms/step - loss: 0.6514 - categorical_accuracy: 0.8379 - val_loss: 2.4324 - val_categorical_accuracy: 0.6476

### Fonction d'activation 
Tanh sigmoid relu

In [14]:
NUM_LAYERS = list(range(3, 6))
BATCH_SIZE = 512
EPOCHS = 100
SHUFFLE = True
learning_rates = 0.001
momentums = 0.5


def ConvNet(num_layer, filters_by_layers):
    input_layer = Input(shape=(32, 32, 3))
    hidden_layers = input_layer
    
    for n in range(num_layer):
        hidden_layers = Conv2D(filters_by_layers[n], (3, 3), padding="same", activation=tanh)(hidden_layers)
        hidden_layers = MaxPool2D()(hidden_layers)
    
    hidden_layers = Flatten()(hidden_layers)
    output_layer = Dense(NUM_CLASSES, activation=softmax)(hidden_layers)
    return Model(input_layer, output_layer)



num_layer = int(np.random.choice(NUM_LAYERS, 1))
filters_by_layers = [32*x for x in range(1, num_layer+1)]

convnet = ConvNet(num_layer, filters_by_layers)
convnet.compile(loss=categorical_crossentropy,
            optimizer=SGD(lr, momentum=mom),
            metrics=categorical_accuracy)
CONVNET_LOG = os.path.join(LOG_DIR, "convnet",
                f"convnet_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_SGD_lr_{lr}_mom_{mom}_layers_{'_'.join(str(e) for e in filters_by_layers)}_af_tanh")
CONVNET_MODEL = os.path.join(MODEL_DIR, "convnet",
                f"convnet_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_SGD_lr_{lr}_mom_{mom}_layers_{'_'.join(str(e) for e in filters_by_layers)}_af_tanh.keras")
convnet.fit(x_train,
            y_train,
            batch_size=BATCH_SIZE,
            epochs=EPOCHS,
            validation_data=(x_test, y_test),
            shuffle=SHUFFLE,
            callbacks=[tf.keras.callbacks.TensorBoard(CONVNET_LOG, histogram_freq=1)] 
           ) 
convnet.save(CONVNET_MODEL)
                       

Epoch 1/100
98/98 [==============================] - 56s 562ms/step - loss: 2.2853 - categorical_accuracy: 0.1413 - val_loss: 2.2574 - val_categorical_accuracy: 0.1575
Epoch 2/100
98/98 [==============================] - 55s 561ms/step - loss: 2.2306 - categorical_accuracy: 0.2011 - val_loss: 2.2008 - val_categorical_accuracy: 0.2361
Epoch 3/100
98/98 [==============================] - 55s 564ms/step - loss: 2.1663 - categorical_accuracy: 0.2501 - val_loss: 2.1267 - val_categorical_accuracy: 0.2889
Epoch 4/100
98/98 [==============================] - 55s 560ms/step - loss: 2.0813 - categorical_accuracy: 0.2905 - val_loss: 2.0295 - val_categorical_accuracy: 0.3120
Epoch 5/100
98/98 [==============================] - 55s 558ms/step - loss: 1.9887 - categorical_accuracy: 0.3107 - val_loss: 1.9450 - val_categorical_accuracy: 0.3314
Epoch 6/100
98/98 [==============================] - 55s 561ms/step - loss: 1.9236 - categorical_accuracy: 0.3271 - val_loss: 1.8901 - val_categorical_accuracy:

98/98 [==============================] - 56s 573ms/step - loss: 1.2852 - categorical_accuracy: 0.5493 - val_loss: 1.2891 - val_categorical_accuracy: 0.5425
Epoch 50/100
98/98 [==============================] - 55s 563ms/step - loss: 1.2793 - categorical_accuracy: 0.5518 - val_loss: 1.2838 - val_categorical_accuracy: 0.5497
Epoch 51/100
98/98 [==============================] - 55s 562ms/step - loss: 1.2709 - categorical_accuracy: 0.5551 - val_loss: 1.2741 - val_categorical_accuracy: 0.5493
Epoch 52/100
98/98 [==============================] - 55s 564ms/step - loss: 1.2633 - categorical_accuracy: 0.5586 - val_loss: 1.2678 - val_categorical_accuracy: 0.5523
Epoch 53/100
98/98 [==============================] - 55s 561ms/step - loss: 1.2568 - categorical_accuracy: 0.5603 - val_loss: 1.2642 - val_categorical_accuracy: 0.5543
Epoch 54/100
98/98 [==============================] - 55s 562ms/step - loss: 1.2497 - categorical_accuracy: 0.5624 - val_loss: 1.2568 - val_categorical_accuracy: 0.5536

98/98 [==============================] - 55s 566ms/step - loss: 1.0340 - categorical_accuracy: 0.6444 - val_loss: 1.0681 - val_categorical_accuracy: 0.6290
Epoch 98/100
98/98 [==============================] - 56s 567ms/step - loss: 1.0298 - categorical_accuracy: 0.6461 - val_loss: 1.0655 - val_categorical_accuracy: 0.6300
Epoch 99/100
98/98 [==============================] - 55s 566ms/step - loss: 1.0249 - categorical_accuracy: 0.6465 - val_loss: 1.0624 - val_categorical_accuracy: 0.6300
Epoch 100/100
98/98 [==============================] - 56s 568ms/step - loss: 1.0223 - categorical_accuracy: 0.6484 - val_loss: 1.0595 - val_categorical_accuracy: 0.6315


In [15]:
NUM_LAYERS = list(range(3, 6))
BATCH_SIZE = 512
EPOCHS = 100
SHUFFLE = True
learning_rates = 0.001
momentums = 0.5


def ConvNet(num_layer, filters_by_layers):
    input_layer = Input(shape=(32, 32, 3))
    hidden_layers = input_layer
    
    for n in range(num_layer):
        hidden_layers = Conv2D(filters_by_layers[n], (3, 3), padding="same", activation=sigmoid)(hidden_layers)
        hidden_layers = MaxPool2D()(hidden_layers)
    
    hidden_layers = Flatten()(hidden_layers)
    output_layer = Dense(NUM_CLASSES, activation=softmax)(hidden_layers)
    return Model(input_layer, output_layer)



num_layer = int(np.random.choice(NUM_LAYERS, 1))
filters_by_layers = [32*x for x in range(1, num_layer+1)]

convnet = ConvNet(num_layer, filters_by_layers)
convnet.compile(loss=categorical_crossentropy,
            optimizer=SGD(lr, momentum=mom),
            metrics=categorical_accuracy)
CONVNET_LOG = os.path.join(LOG_DIR, "convnet",
                f"convnet_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_SGD_lr_{lr}_mom_{mom}_layers_{'_'.join(str(e) for e in filters_by_layers)}_af_sigmoid")
CONVNET_MODEL = os.path.join(MODEL_DIR, "convnet",
                f"convnet_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_SGD_lr_{lr}_mom_{mom}_layers_{'_'.join(str(e) for e in filters_by_layers)}_af_sigmoid.keras")
convnet.fit(x_train,
            y_train,
            batch_size=BATCH_SIZE,
            epochs=EPOCHS,
            validation_data=(x_test, y_test),
            shuffle=SHUFFLE,
            callbacks=[tf.keras.callbacks.TensorBoard(CONVNET_LOG, histogram_freq=1)] 
           ) 
convnet.save(CONVNET_MODEL)
                       

Epoch 1/100
98/98 [==============================] - 63s 637ms/step - loss: 2.3116 - categorical_accuracy: 0.0990 - val_loss: 2.3032 - val_categorical_accuracy: 0.1000
Epoch 2/100
98/98 [==============================] - 62s 638ms/step - loss: 2.3034 - categorical_accuracy: 0.0989 - val_loss: 2.3032 - val_categorical_accuracy: 0.1000
Epoch 3/100
98/98 [==============================] - 63s 642ms/step - loss: 2.3032 - categorical_accuracy: 0.0994 - val_loss: 2.3031 - val_categorical_accuracy: 0.1000
Epoch 4/100
98/98 [==============================] - 63s 644ms/step - loss: 2.3035 - categorical_accuracy: 0.0983 - val_loss: 2.3031 - val_categorical_accuracy: 0.1000
Epoch 5/100
98/98 [==============================] - 62s 635ms/step - loss: 2.3034 - categorical_accuracy: 0.0997 - val_loss: 2.3030 - val_categorical_accuracy: 0.1000
Epoch 6/100
98/98 [==============================] - 62s 635ms/step - loss: 2.3036 - categorical_accuracy: 0.0979 - val_loss: 2.3032 - val_categorical_accuracy:

98/98 [==============================] - 62s 636ms/step - loss: 2.3032 - categorical_accuracy: 0.1006 - val_loss: 2.3029 - val_categorical_accuracy: 0.1000
Epoch 50/100
98/98 [==============================] - 62s 636ms/step - loss: 2.3033 - categorical_accuracy: 0.0988 - val_loss: 2.3032 - val_categorical_accuracy: 0.1000
Epoch 51/100
98/98 [==============================] - 62s 633ms/step - loss: 2.3034 - categorical_accuracy: 0.0987 - val_loss: 2.3035 - val_categorical_accuracy: 0.1000
Epoch 52/100
98/98 [==============================] - 62s 636ms/step - loss: 2.3034 - categorical_accuracy: 0.0994 - val_loss: 2.3032 - val_categorical_accuracy: 0.1000
Epoch 53/100
98/98 [==============================] - 62s 634ms/step - loss: 2.3036 - categorical_accuracy: 0.0997 - val_loss: 2.3031 - val_categorical_accuracy: 0.1000
Epoch 54/100
98/98 [==============================] - 62s 635ms/step - loss: 2.3033 - categorical_accuracy: 0.1002 - val_loss: 2.3032 - val_categorical_accuracy: 0.1000

98/98 [==============================] - 62s 637ms/step - loss: 2.3032 - categorical_accuracy: 0.0979 - val_loss: 2.3030 - val_categorical_accuracy: 0.1000
Epoch 98/100
98/98 [==============================] - 62s 636ms/step - loss: 2.3033 - categorical_accuracy: 0.0985 - val_loss: 2.3035 - val_categorical_accuracy: 0.1000
Epoch 99/100
98/98 [==============================] - 62s 635ms/step - loss: 2.3031 - categorical_accuracy: 0.1017 - val_loss: 2.3030 - val_categorical_accuracy: 0.1000
Epoch 100/100
98/98 [==============================] - 62s 636ms/step - loss: 2.3033 - categorical_accuracy: 0.1013 - val_loss: 2.3032 - val_categorical_accuracy: 0.1000


In [16]:
NUM_LAYERS = list(range(3, 6))
BATCH_SIZE = 512
EPOCHS = 100
SHUFFLE = True
learning_rates = 0.001
momentums = 0.5


def ConvNet(num_layer, filters_by_layers):
    input_layer = Input(shape=(32, 32, 3))
    hidden_layers = input_layer
    
    for n in range(num_layer):
        hidden_layers = Conv2D(filters_by_layers[n], (3, 3), padding="same", activation=relu)(hidden_layers)
        hidden_layers = MaxPool2D()(hidden_layers)
    
    hidden_layers = Flatten()(hidden_layers)
    output_layer = Dense(NUM_CLASSES, activation=softmax)(hidden_layers)
    return Model(input_layer, output_layer)



num_layer = int(np.random.choice(NUM_LAYERS, 1))
filters_by_layers = [32*x for x in range(1, num_layer+1)]

convnet = ConvNet(num_layer, filters_by_layers)
convnet.compile(loss=categorical_crossentropy,
            optimizer=SGD(lr, momentum=mom),
            metrics=categorical_accuracy)
CONVNET_LOG = os.path.join(LOG_DIR, "convnet",
                f"convnet_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_SGD_lr_{lr}_mom_{mom}_layers_{'_'.join(str(e) for e in filters_by_layers)}_af_relu")
CONVNET_MODEL = os.path.join(MODEL_DIR, "convnet",
                f"convnet_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_SGD_lr_{lr}_mom_{mom}_layers_{'_'.join(str(e) for e in filters_by_layers)}_af_relu.keras")
convnet.fit(x_train,
            y_train,
            batch_size=BATCH_SIZE,
            epochs=EPOCHS,
            validation_data=(x_test, y_test),
            shuffle=SHUFFLE,
            callbacks=[tf.keras.callbacks.TensorBoard(CONVNET_LOG, histogram_freq=1)] 
           ) 
convnet.save(CONVNET_MODEL)
                       

Epoch 1/100
98/98 [==============================] - 60s 612ms/step - loss: 2.3004 - categorical_accuracy: 0.1095 - val_loss: 2.2953 - val_categorical_accuracy: 0.1233
Epoch 2/100
98/98 [==============================] - 60s 608ms/step - loss: 2.2913 - categorical_accuracy: 0.1324 - val_loss: 2.2863 - val_categorical_accuracy: 0.1454
Epoch 3/100
98/98 [==============================] - 59s 607ms/step - loss: 2.2800 - categorical_accuracy: 0.1639 - val_loss: 2.2716 - val_categorical_accuracy: 0.1732
Epoch 4/100
98/98 [==============================] - 60s 611ms/step - loss: 2.2597 - categorical_accuracy: 0.1914 - val_loss: 2.2438 - val_categorical_accuracy: 0.2023
Epoch 5/100
98/98 [==============================] - 61s 627ms/step - loss: 2.2184 - categorical_accuracy: 0.2085 - val_loss: 2.1844 - val_categorical_accuracy: 0.2200
Epoch 6/100
98/98 [==============================] - 60s 615ms/step - loss: 2.1400 - categorical_accuracy: 0.2308 - val_loss: 2.0894 - val_categorical_accuracy:

98/98 [==============================] - 59s 607ms/step - loss: 1.2303 - categorical_accuracy: 0.5705 - val_loss: 1.2581 - val_categorical_accuracy: 0.5537
Epoch 50/100
98/98 [==============================] - 60s 611ms/step - loss: 1.2168 - categorical_accuracy: 0.5733 - val_loss: 1.2582 - val_categorical_accuracy: 0.5528
Epoch 51/100
98/98 [==============================] - 60s 609ms/step - loss: 1.2160 - categorical_accuracy: 0.5758 - val_loss: 1.2474 - val_categorical_accuracy: 0.5553
Epoch 52/100
98/98 [==============================] - 59s 607ms/step - loss: 1.2077 - categorical_accuracy: 0.5775 - val_loss: 1.2664 - val_categorical_accuracy: 0.5482
Epoch 53/100
98/98 [==============================] - 59s 607ms/step - loss: 1.2241 - categorical_accuracy: 0.5724 - val_loss: 1.2796 - val_categorical_accuracy: 0.5425
Epoch 54/100
98/98 [==============================] - 59s 606ms/step - loss: 1.1958 - categorical_accuracy: 0.5828 - val_loss: 1.2243 - val_categorical_accuracy: 0.5620

98/98 [==============================] - 59s 607ms/step - loss: 0.9220 - categorical_accuracy: 0.6843 - val_loss: 1.0848 - val_categorical_accuracy: 0.6242
Epoch 98/100
98/98 [==============================] - 59s 606ms/step - loss: 0.9205 - categorical_accuracy: 0.6814 - val_loss: 1.0618 - val_categorical_accuracy: 0.6314
Epoch 99/100
98/98 [==============================] - 59s 607ms/step - loss: 0.9103 - categorical_accuracy: 0.6871 - val_loss: 1.0653 - val_categorical_accuracy: 0.6290
Epoch 100/100
98/98 [==============================] - 59s 607ms/step - loss: 0.9061 - categorical_accuracy: 0.6871 - val_loss: 1.0636 - val_categorical_accuracy: 0.6320


### Structure Pooling

In [24]:
NUM_LAYERS = list(range(3, 6))
BATCH_SIZE = 512
EPOCHS = 100
SHUFFLE = True
learning_rates = 0.001
momentums = 0.5


def ConvNet(num_layer, filters_by_layers):
    input_layer = Input(shape=(32, 32, 3))
    hidden_layers = input_layer
    
    for n in range(num_layer):
        hidden_layers = Conv2D(filters_by_layers[n], (3, 3), padding="same", activation=tanh)(hidden_layers)
        hidden_layers = tf.keras.layers.AveragePooling2D()(hidden_layers)
    
    hidden_layers = Flatten()(hidden_layers)
    output_layer = Dense(NUM_CLASSES, activation=softmax)(hidden_layers)
    return Model(input_layer, output_layer)


num_layer = int(np.random.choice(NUM_LAYERS, 1))
filters_by_layers = [32*x for x in range(1, num_layer+1)]

convnet = ConvNet(num_layer, filters_by_layers)
convnet.compile(loss=categorical_crossentropy,
            optimizer=SGD(lr, momentum=mom),
            metrics=categorical_accuracy)
CONVNET_LOG = os.path.join(LOG_DIR, "convnet",
                f"convnet_ep_{EPOCHS}_bs_{BATCH_SIZE}_AVGPOOLING_opt_SGD_lr_{lr}_mom_{mom}_layers_{'_'.join(str(e) for e in filters_by_layers)}_af_tanh")
CONVNET_MODEL = os.path.join(MODEL_DIR, "convnet",
                f"convnet_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_SGD_lr_{lr}_mom_{mom}_layers_{'_'.join(str(e) for e in filters_by_layers)}_af_tanh.keras")
convnet.fit(x_train,
            y_train,
            batch_size=BATCH_SIZE,
            epochs=EPOCHS,
            validation_data=(x_test, y_test),
            shuffle=SHUFFLE,
            callbacks=[tf.keras.callbacks.TensorBoard(CONVNET_LOG, histogram_freq=1)]
           ) 
convnet.save(CONVNET_MODEL)

Epoch 1/100
98/98 [==============================] - 55s 551ms/step - loss: 2.2908 - categorical_accuracy: 0.1195 - val_loss: 2.2659 - val_categorical_accuracy: 0.1405
Epoch 2/100
98/98 [==============================] - 51s 521ms/step - loss: 2.2447 - categorical_accuracy: 0.1744 - val_loss: 2.2170 - val_categorical_accuracy: 0.2245
Epoch 3/100
98/98 [==============================] - 51s 525ms/step - loss: 2.1838 - categorical_accuracy: 0.2346 - val_loss: 2.1405 - val_categorical_accuracy: 0.2635
Epoch 4/100
98/98 [==============================] - 53s 544ms/step - loss: 2.0987 - categorical_accuracy: 0.2603 - val_loss: 2.0514 - val_categorical_accuracy: 0.2778
Epoch 5/100
98/98 [==============================] - 54s 556ms/step - loss: 2.0321 - categorical_accuracy: 0.2779 - val_loss: 2.0050 - val_categorical_accuracy: 0.2978
Epoch 6/100
98/98 [==============================] - 51s 516ms/step - loss: 2.0011 - categorical_accuracy: 0.2890 - val_loss: 1.9830 - val_categorical_accuracy:

98/98 [==============================] - 50s 510ms/step - loss: 1.6485 - categorical_accuracy: 0.4230 - val_loss: 1.6461 - val_categorical_accuracy: 0.4198
Epoch 50/100
98/98 [==============================] - 51s 520ms/step - loss: 1.6448 - categorical_accuracy: 0.4256 - val_loss: 1.6391 - val_categorical_accuracy: 0.4226
Epoch 51/100
98/98 [==============================] - 50s 509ms/step - loss: 1.6413 - categorical_accuracy: 0.4271 - val_loss: 1.6352 - val_categorical_accuracy: 0.4250
Epoch 52/100
98/98 [==============================] - 49s 503ms/step - loss: 1.6371 - categorical_accuracy: 0.4280 - val_loss: 1.6313 - val_categorical_accuracy: 0.4261
Epoch 53/100
98/98 [==============================] - 49s 504ms/step - loss: 1.6341 - categorical_accuracy: 0.4289 - val_loss: 1.6281 - val_categorical_accuracy: 0.4266
Epoch 54/100
98/98 [==============================] - 49s 503ms/step - loss: 1.6304 - categorical_accuracy: 0.4307 - val_loss: 1.6251 - val_categorical_accuracy: 0.4255

98/98 [==============================] - 49s 505ms/step - loss: 1.4654 - categorical_accuracy: 0.4881 - val_loss: 1.4653 - val_categorical_accuracy: 0.4810
Epoch 98/100
98/98 [==============================] - 49s 503ms/step - loss: 1.4614 - categorical_accuracy: 0.4877 - val_loss: 1.4600 - val_categorical_accuracy: 0.4840
Epoch 99/100
98/98 [==============================] - 50s 506ms/step - loss: 1.4570 - categorical_accuracy: 0.4898 - val_loss: 1.4563 - val_categorical_accuracy: 0.4838
Epoch 100/100
98/98 [==============================] - 49s 504ms/step - loss: 1.4539 - categorical_accuracy: 0.4914 - val_loss: 1.4551 - val_categorical_accuracy: 0.4848


In [27]:
BATCH_SIZE = 512
EPOCHS = 100
SHUFFLE = True
learning_rates = 0.001
momentums = 0.9


convnet = Sequential() 
convnet.add(Conv2D(128,input_shape=(32, 32, 3), kernel_size=(3, 3), padding="same", activation=tanh)) 
convnet.add(Conv2D(64,kernel_size=(3, 3), padding="same", activation=tanh))  
convnet.add(Conv2D(32,kernel_size=(3, 3), padding="same", activation=tanh)) 
convnet.add(MaxPool2D())  
convnet.add(Flatten())
convnet.add(Dense(NUM_CLASSES, activation=softmax))

convnet.compile(loss=categorical_crossentropy,
            optimizer=SGD(lr, momentum=mom),
            metrics=categorical_accuracy)
CONVNET_LOG = os.path.join(LOG_DIR, "convnet",
                f"convnet_sequential_ep_{EPOCHS}_bs_{BATCH_SIZE}_MAXPOOLING_opt_SGD_lr_{lr}_mom_{mom}_layers_{'_'.join(str(e) for e in filters_by_layers)}_af_tanh")
CONVNET_MODEL = os.path.join(MODEL_DIR, "convnet",
                f"convnet__sequential_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_SGD_lr_{lr}_mom_{mom}_layers_{'_'.join(str(e) for e in filters_by_layers)}_af_tanh.keras")
convnet.fit(x_train,
            y_train,
            batch_size=BATCH_SIZE,
            epochs=EPOCHS,
            validation_data=(x_test, y_test),
            shuffle=SHUFFLE,
            callbacks=[tf.keras.callbacks.TensorBoard(CONVNET_LOG, histogram_freq=1)]
           ) 
convnet.save(CONVNET_MODEL)

Epoch 1/100
98/98 [==============================] - 455s 5s/step - loss: 2.2372 - categorical_accuracy: 0.1812 - val_loss: 2.1348 - val_categorical_accuracy: 0.2443
Epoch 2/100
98/98 [==============================] - 452s 5s/step - loss: 2.0194 - categorical_accuracy: 0.2963 - val_loss: 1.9375 - val_categorical_accuracy: 0.3244
Epoch 3/100
98/98 [==============================] - 450s 5s/step - loss: 1.9067 - categorical_accuracy: 0.3372 - val_loss: 1.8739 - val_categorical_accuracy: 0.3518
Epoch 4/100
98/98 [==============================] - 451s 5s/step - loss: 1.8429 - categorical_accuracy: 0.3622 - val_loss: 1.8047 - val_categorical_accuracy: 0.3763
Epoch 5/100
98/98 [==============================] - 452s 5s/step - loss: 1.7864 - categorical_accuracy: 0.3838 - val_loss: 1.7549 - val_categorical_accuracy: 0.3921
Epoch 6/100
98/98 [==============================] - 452s 5s/step - loss: 1.7419 - categorical_accuracy: 0.3969 - val_loss: 1.7171 - val_categorical_accuracy: 0.4061
Epoc

98/98 [==============================] - 453s 5s/step - loss: 1.0355 - categorical_accuracy: 0.6434 - val_loss: 1.1286 - val_categorical_accuracy: 0.6037
Epoch 51/100
98/98 [==============================] - 452s 5s/step - loss: 1.0243 - categorical_accuracy: 0.6473 - val_loss: 1.1147 - val_categorical_accuracy: 0.6068
Epoch 52/100
98/98 [==============================] - 452s 5s/step - loss: 1.0175 - categorical_accuracy: 0.6506 - val_loss: 1.1157 - val_categorical_accuracy: 0.6027
Epoch 53/100
98/98 [==============================] - 452s 5s/step - loss: 1.0104 - categorical_accuracy: 0.6548 - val_loss: 1.1077 - val_categorical_accuracy: 0.6094
Epoch 54/100
98/98 [==============================] - 453s 5s/step - loss: 1.0012 - categorical_accuracy: 0.6558 - val_loss: 1.1018 - val_categorical_accuracy: 0.6091
Epoch 55/100
98/98 [==============================] - 453s 5s/step - loss: 0.9967 - categorical_accuracy: 0.6569 - val_loss: 1.0968 - val_categorical_accuracy: 0.6138
Epoch 56/10

98/98 [==============================] - 453s 5s/step - loss: 0.7539 - categorical_accuracy: 0.7440 - val_loss: 0.9782 - val_categorical_accuracy: 0.6611
Epoch 100/100
98/98 [==============================] - 453s 5s/step - loss: 0.7486 - categorical_accuracy: 0.7467 - val_loss: 0.9794 - val_categorical_accuracy: 0.6593


In [28]:
NUM_LAYERS = list(range(3, 6))
BATCH_SIZE = 512
EPOCHS = 100
SHUFFLE = True
learning_rates = 0.001
momentums = 0.5


def ConvNet(num_layer, filters_by_layers):
    input_layer = Input(shape=(32, 32, 3))
    hidden_layers = input_layer
    
    for n in range(num_layer):
        hidden_layers = Conv2D(filters_by_layers[n], (3, 3), padding="valid", activation=relu)(hidden_layers)
        hidden_layers = MaxPool2D()(hidden_layers)
    
    hidden_layers = Flatten()(hidden_layers)
    output_layer = Dense(NUM_CLASSES, activation=softmax)(hidden_layers)
    return Model(input_layer, output_layer)



num_layer = int(np.random.choice(NUM_LAYERS, 1))
filters_by_layers = [32*x for x in range(1, num_layer+1)]

convnet = ConvNet(num_layer, filters_by_layers)
convnet.compile(loss=categorical_crossentropy,
            optimizer=SGD(lr, momentum=mom),
            metrics=categorical_accuracy)
CONVNET_LOG = os.path.join(LOG_DIR, "convnet",
                f"convnet_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_SGD_lr_{lr}_mom_{mom}_layers_{'_'.join(str(e) for e in filters_by_layers)}_af_relu")
CONVNET_MODEL = os.path.join(MODEL_DIR, "convnet",
                f"convnet_ep_{EPOCHS}_bs_{BATCH_SIZE}_opt_SGD_lr_{lr}_mom_{mom}_layers_{'_'.join(str(e) for e in filters_by_layers)}_af_relu.keras")
convnet.fit(x_train,
            y_train,
            batch_size=BATCH_SIZE,
            epochs=EPOCHS,
            validation_data=(x_test, y_test),
            shuffle=SHUFFLE,
            callbacks=[tf.keras.callbacks.TensorBoard(CONVNET_LOG, histogram_freq=1)] 
           ) 
convnet.save(CONVNET_MODEL)

Epoch 1/100
98/98 [==============================] - 35s 347ms/step - loss: 2.3006 - categorical_accuracy: 0.1061 - val_loss: 2.2946 - val_categorical_accuracy: 0.1194
Epoch 2/100
98/98 [==============================] - 34s 345ms/step - loss: 2.2899 - categorical_accuracy: 0.1279 - val_loss: 2.2847 - val_categorical_accuracy: 0.1379
Epoch 3/100
98/98 [==============================] - 34s 347ms/step - loss: 2.2789 - categorical_accuracy: 0.1593 - val_loss: 2.2714 - val_categorical_accuracy: 0.1613
Epoch 4/100
98/98 [==============================] - 34s 346ms/step - loss: 2.2624 - categorical_accuracy: 0.1880 - val_loss: 2.2505 - val_categorical_accuracy: 0.2032
Epoch 5/100
98/98 [==============================] - 34s 345ms/step - loss: 2.2350 - categorical_accuracy: 0.2189 - val_loss: 2.2135 - val_categorical_accuracy: 0.2362
Epoch 6/100
98/98 [==============================] - 34s 346ms/step - loss: 2.1872 - categorical_accuracy: 0.2423 - val_loss: 2.1517 - val_categorical_accuracy:

98/98 [==============================] - 34s 344ms/step - loss: 1.3872 - categorical_accuracy: 0.5103 - val_loss: 1.3928 - val_categorical_accuracy: 0.5079
Epoch 50/100
98/98 [==============================] - 34s 347ms/step - loss: 1.3782 - categorical_accuracy: 0.5128 - val_loss: 1.4068 - val_categorical_accuracy: 0.4975
Epoch 51/100
98/98 [==============================] - 34s 347ms/step - loss: 1.3710 - categorical_accuracy: 0.5171 - val_loss: 1.3878 - val_categorical_accuracy: 0.5106
Epoch 52/100
98/98 [==============================] - 34s 345ms/step - loss: 1.3659 - categorical_accuracy: 0.5188 - val_loss: 1.4168 - val_categorical_accuracy: 0.5023
Epoch 53/100
98/98 [==============================] - 34s 346ms/step - loss: 1.3642 - categorical_accuracy: 0.5186 - val_loss: 1.3721 - val_categorical_accuracy: 0.5195
Epoch 54/100
98/98 [==============================] - 34s 347ms/step - loss: 1.3538 - categorical_accuracy: 0.5224 - val_loss: 1.3707 - val_categorical_accuracy: 0.5192

98/98 [==============================] - 34s 347ms/step - loss: 1.1490 - categorical_accuracy: 0.6013 - val_loss: 1.2026 - val_categorical_accuracy: 0.5773
Epoch 98/100
98/98 [==============================] - 34s 345ms/step - loss: 1.1452 - categorical_accuracy: 0.6013 - val_loss: 1.2157 - val_categorical_accuracy: 0.5697
Epoch 99/100
98/98 [==============================] - 34s 346ms/step - loss: 1.1391 - categorical_accuracy: 0.6050 - val_loss: 1.1863 - val_categorical_accuracy: 0.5849
Epoch 100/100
98/98 [==============================] - 34s 347ms/step - loss: 1.1313 - categorical_accuracy: 0.6089 - val_loss: 1.1946 - val_categorical_accuracy: 0.5845
